In [1]:
import requests
from bs4 import BeautifulSoup
import json
import re

In [35]:
# Формирование исходных URL
base_url = "http://books.toscrape.com/"
url = base_url + "catalogue/page-1.html"

In [36]:
pages_counter = 0 ### для подсчета количества обработанных веб-страниц
books = [] ### для сохранения распарсенных книг 

In [37]:
while url:
    print("\nПроводится скрапинг страницы №", pages_counter+1)
    # Отправка GET запроса по URL
    response = requests.get(url)

    # Парсинг HTML страницы с использованием BeautifulSoup
    soup = BeautifulSoup(response.text, "html.parser")

    h3_tags = soup.find_all("h3")

    for h3_tag in h3_tags:

        # Поиск тега <a> 
        a_tag = h3_tag.find("a", href=True)

        # Создание ссылки на страницу книги
        book_url = base_url + "catalogue/" + a_tag["href"]

        # Отправка запроса по ссылке на книгу
        book_response = requests.get(book_url)

        # Парсинг страницы книги
        book_soup = BeautifulSoup(book_response.text, "html.parser")

        ### экстракция требуемых данных о книге
        title = book_soup.find("h1").text.strip()
        price = float(book_soup.find("p", class_="price_color").text.strip().replace("Â\u00a3", ""))
        stock = int(re.findall(r'\d+', book_soup.find("p", class_="instock availability").text.strip())[0])
        description = book_soup.find("meta", attrs={"name": "description"})["content"]

        # Добавление информации о книге в общий список
        books.append({
                "Название": title,
                "Цена в фунтах стерлингов": price,
                "Количество в наличии": stock,
                "Описание": description
            })
        print("Добавили книгу", title)
    
    next_button = soup.find('a', string='next')
        
    ### Проверка, есть ли ссылка "next" на следующую страницу
    if next_button:
        url = base_url + "catalogue/" + next_button['href']
        pages_counter += 1
    else:
        url = None  # если ссылки нет - выходим из цикла while


Проводится скрапинг страницы № 1
Добавили книгу A Light in the Attic
Добавили книгу Tipping the Velvet
Добавили книгу Soumission
Добавили книгу Sharp Objects
Добавили книгу Sapiens: A Brief History of Humankind
Добавили книгу The Requiem Red
Добавили книгу The Dirty Little Secrets of Getting Your Dream Job
Добавили книгу The Coming Woman: A Novel Based on the Life of the Infamous Feminist, Victoria Woodhull
Добавили книгу The Boys in the Boat: Nine Americans and Their Epic Quest for Gold at the 1936 Berlin Olympics
Добавили книгу The Black Maria
Добавили книгу Starving Hearts (Triangular Trade Trilogy, #1)
Добавили книгу Shakespeare's Sonnets
Добавили книгу Set Me Free
Добавили книгу Scott Pilgrim's Precious Little Life (Scott Pilgrim #1)
Добавили книгу Rip it Up and Start Again
Добавили книгу Our Band Could Be Your Life: Scenes from the American Indie Underground, 1981-1991
Добавили книгу Olio
Добавили книгу Mesaerion: The Best Science Fiction Stories 1800-1849
Добавили книгу Liberta

In [38]:
print("\nСуммарное число книг: ", len(books)) 
### результат расчета суммарного числа распарсенных книг соответствует указанному на сайте


Суммарное число книг:  1000


In [39]:
### сохранение общего списка в файл кодировкой, содержащей буквы русского алфавита
with open("books_from_books.toscrape.com.json", "w", encoding='utf-8') as f:
        json.dump(books, f, indent=4, ensure_ascii=False)